In [1]:
from torch import manual_seed, tensor, nonzero, logical_not, load, save, flatten
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split, IterableDataset, Dataset, sampler
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation,Resize, Grayscale
from torchvision.datasets.mnist import MNIST 
from torchshow import show
from CNN_setup.model.CIFAR_CNN import CIFAR_CNN_Classifier
from CNN_setup.vars.CIFARvars import CIFAR10_classes
from CNN_setup.datasets.datasets import CustomCIFAR10, CustomMNIST

from CNN_setup.model.MNIST_CNN import Mnist_CNN_Classifier
from CNN_setup.vars.MNISTvars import MNIST_classes

from CNN_setup.utils.cnn_models_utils import load_model, evaluate

from PIL import Image
from torchshow import show

from CNN_setup.datasets.dataset_tools import save_dataset, combine_datasets, load_dataset

from torchvision.datasets import ImageFolder

from torch.utils.data import ConcatDataset

# Incremental, abrupt, and transformations

In [3]:
transform = Compose([ToTensor()])

In [4]:
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CIFAR10(root='./data', train=False, download=False, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32)

Files already downloaded and verified


### Rotate cifar

In [5]:
raw_data = CIFAR10(root='./data', train=False, download=True)

Files already downloaded and verified


In [6]:
rotated = ImageFolder(root='data/transformed/cifar-rotated90', transform=ToTensor())
dataloader_rotated = DataLoader(dataset=rotated, batch_size = 32)

In [7]:
model = load_model('trained_models\CNN_cifar_downloaded.torch', CIFAR_CNN_Classifier())

In [8]:
evaluate(model=model, test_dataloader=dataloader_rotated, classes=CIFAR10_classes)

Accuracy for class: plane is 38.2 %
Accuracy for class: car is 0.8 %
Accuracy for class: bird is 8.7 %
Accuracy for class: cat is 8.1 %
Accuracy for class: deer is 66.8 %
Accuracy for class: dog is 3.0 %
Accuracy for class: frog is 11.9 %
Accuracy for class: horse is 0.9 %
Accuracy for class: ship is 18.6 %
Accuracy for class: truck is 11.0 %


### Check MNIST rotate

In [ ]:
transform_rotate = Compose([ToTensor(), RandomRotation(degrees=(90,90))])

In [ ]:
raw_data_mnist = MNIST(root='./data', train=False, download=True)

In [ ]:
save_dataset(path='data/transformed/mnist-rotated90', data=raw_data_mnist, transform=Image.Image.rotate, args=(90,))
save_dataset(path='data/transformed/mnist-wo-0', data=raw_data_mnist)

In [3]:
raw_data_mnist = MNIST(root='./data', train=True, download=False)
save_dataset(path='data/transformed/train_mnist/mnist-wo-0', data=raw_data_mnist)

In [ ]:
# CustomerMnist_obj = CustomMNIST(root='./data', train=True, download=True, transform=transform_rotate) ## Another way of creating fully rotated dataset
# CustomerMnist_obj_dataloader = DataLoader(CustomerMnist_obj, batch_size=32)

MNIST_model = load_model("trained_models/CNN_mnist_downloaded.torch",Mnist_CNN_Classifier())

In [ ]:
rotated_mnist = ImageFolder(root='data/transformed/mnist-rotated90', transform=Compose([ToTensor(),Grayscale(num_output_channels=1)]))
dataloader_rotated_mnist = DataLoader(dataset=rotated_mnist, batch_size = 32)

In [ ]:
evaluate(dataloader_rotated_mnist,MNIST_model,classes = MNIST_classes)

Accuracy for class: 0 is 56.0 %
Accuracy for class: 1 is 0.8 %
Accuracy for class: 2 is 21.0 %
Accuracy for class: 3 is 0.7 %
Accuracy for class: 4 is 9.4 %
Accuracy for class: 5 is 8.3 %
Accuracy for class: 6 is 15.6 %
Accuracy for class: 7 is 9.0 %
Accuracy for class: 8 is 10.2 %
Accuracy for class: 9 is 16.7 %
Total Accuracy: 14.6 %


## Incremental

In [7]:
from torch.utils.data import Dataset, Subset

In [17]:
def combine_datasets(dataset1:Dataset, dataset2:Dataset, proportion1:float):
    assert 1 >= proportion1 > 0 
    proportion2 = 1 - proportion1
    # Calculate the total number of samples based on the proportions
    total_samples = int((len(dataset1) * proportion1) + (len(dataset2) * proportion2))

    # Calculate the number of samples to take from each dataset
    threshold1 = int(total_samples * proportion1)
    threshold2 = int(total_samples * proportion2)

    # Take the first 'threshold' samples from each dataset
    subset1 = Subset(dataset1, range(threshold1))
    subset2 = Subset(dataset2, range(threshold2))
    
    # Combine the two datasets
    combined_data = ConcatDataset([subset1, subset2])
    
    return combined_data

x = combine_datasets(rotated,raw_data,proportion1=1)
x

In [18]:
for i, l in x:
    break
l

0

In [2]:
test = load_dataset(r'data\transformed\cifar-only-0',flat=True)
test_dataloader = DataLoader(test,batch_size=32)

In [3]:
from incremental_ks.IncrementalKS.IKSSW import IKSSW, IKS
import random
from collections import deque

In [ ]:
def detect_drift_iks(df_train, df_test):

  iks_statistics = [] # collect statistics generated by IKS
  iks = IKS() # group 0 = reference; group 1 = sliding

  sliding = deque()

  for _, row in df_train.iterrows():
    iks.Add((row, random()), 0)
    wrnd = (row, random()) # we only need to keep RND component for values in the sliding window
    iks.Add(wrnd, 1)
    sliding.append(wrnd)

  # process sliding window
  for _, row in df_test.iterrows():
    iks.Remove(sliding.popleft(), 1)
    wrnd = (row, random())
    iks.Add(wrnd, 1)
    sliding.append(wrnd)

    iks_statistics.append(iks.KS())
  
  return iks_statistics
